In [1]:
from math import sqrt
from numpy import concatenate
import numpy as np
from matplotlib import pyplot
import os

import pandas as pd

from scipy.stats import uniform

from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error

from keras.models import Sequential, Model
from keras.layers import Activation, Input, Dense, Conv1D, MaxPooling1D, AveragePooling1D, Dropout, Flatten, GRU, LSTM
from keras.layers.advanced_activations import ELU
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.optimizers import *
from keras import backend as K
K.set_image_dim_ordering('th')

import tensorflow as tf
from pymlx import *

import warnings
warnings.filterwarnings('ignore')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# from IPython.display import Audio

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
tf.__version__

'1.5.0'

In [3]:
def make_windowed(dataset, seq_length, horizon, feat_cols, resp_cols, region_col, split=0.2, resp_width=0,
                        predict_current=False):
    """
    This function takes data and creates a windowed dataframe to be used in time-series analysis. 
    dataset: [panda df] the raw dataframe that a time series windowed df will be created from
    seq_length: [int] the sequence length, the amount of time to be used to perform the 
                time-series analysis
    horizon: [int] how far into the future you wish to predict
    feat_cols: [list] a list of column names that make up the feature space
    resp_cols: [list] a list of column names that make up the response
    region_col: [str] the name of the column that different time-series will be created on, i.e. 
                different regions that contain
                independent time-series.
    split: [float] A percent split for the test set, i.e. 0.2 equals a 80/20 split for the 
                train/test sets.
    resp_width: [int] If you want to predict out to a set distance you set the horizon to that 
                    time point and this value to 0, however if you want to predict every value 
                    between let's say now and some point in the future you set horizon to 1 and
                    the resp_width to that point. The algorithm will then predict every time point.
    predict_current: [bool] horizon needs to be set to 1 for this to work. This will predict at 
                    the current time so if there is a sequence length of 2 instead of forecasting 
                    out the horizon length, the model
                    will predict at the current time.
    """

    if ((predict_current) and (horizon is not 1)):
        raise ValueError('If predict_current is set to True, then Horizon must be set to 1.')
    else:
        pass

    if (any(dataset[feat_cols + resp_cols].isnull().sum()) != 0):
        raise ValueError('There is missing data in at least one of the columns supplied in keep_cols. \
                         Please impute this missing data as needed.')
    else:
        pass

    # check to see if there are same features in both the response and the features list
    resp_and_feats = [var for var in feat_cols if var in resp_cols]

    regions = list(dataset[region_col].unique())
    big_dict = {}

    for i in range(len(regions)):
        big_dict.update({regions[i]: dataset[dataset[region_col] == regions[i]]})
    features = len(feat_cols)
    response = len(resp_cols)
    if resp_width == 0:
        train_X_all, test_X_all = np.empty((0, seq_length, features)), np.empty((0, seq_length, features))
        train_y_all, test_y_all = np.empty((0, response)), np.empty((0, response))
    else:
        train_X_all, test_X_all = np.empty((0, sequence_length, features)), np.empty((0, sequence_length, features))
        train_y_all, test_y_all = np.empty((0, resp_width, response)), np.empty((0, resp_width, response))

    for region in regions:
        big_dict[region] = big_dict[region][resp_cols + feat_cols]
        if resp_and_feats:
            big_dict[region] = big_dict[region].loc[:, ~big_dict[region].columns.duplicated()]
        else:
            pass

        length = len(big_dict[region])
        test_length = int(length * split)  # 20% test set

        df_x = big_dict[region][feat_cols]
        df_y = big_dict[region][resp_cols]
        ts_x = df_x.values
        ts_y = df_y.values

        train_length = length - test_length

        train_X, train_y, test_X, test_y = [], [], [], []

        if (predict_current):
            z = 2
            q = 1
        else:
            z = 1
            q = 0

        if (resp_width != 0):
            for i in range(train_length - sequence_length - horizon - resp_width):
                train_X.append(ts_x[i:i + sequence_length])
                train_y.append(ts_y[i + sequence_length + horizon - z:i + sequence_length + horizon - z + resp_width])
            for i in range(-test_length, -resp_width):
                test_X.append(ts_x[i - sequence_length - horizon + 1:i - horizon + 1])
                test_y.append(ts_y[i - q:i - q + (resp_width)])

        else:
            for i in range(train_length - seq_length - horizon):
                train_X.append(ts_x[i:i + seq_length])
                train_y.append(ts_y[i + seq_length + horizon - z])
            for i in range(-test_length, 0):
                test_X.append(ts_x[i - seq_length - horizon + 1:i - horizon + 1])
                test_y.append(ts_y[i - q])

        train_X, train_y = np.array(train_X), np.array(train_y)
        test_X, test_y = np.array(test_X), np.array(test_y)

        train_X_all = np.append(train_X_all, train_X, axis=0)
        test_X_all = np.append(test_X_all, test_X, axis=0)
        train_y_all = np.append(train_y_all, train_y, axis=0)
        test_y_all = np.append(test_y_all, test_y, axis=0)

    # normalize
    mean_x = train_X_all.mean(0)
    std_x = train_X_all.std(0)

    train_X = (train_X_all - mean_x) / std_x
    test_X = (test_X_all - mean_x) / std_x
    train_X = train_X.astype('float32')
    test_X = test_X.astype('float32')

    mean_y = train_y_all.mean(0)
    std_y = train_y_all.std(0)

    train_y = (train_y_all - mean_y) / std_y
    test_y = (test_y_all - mean_y) / std_y
    train_y = train_y.astype('float32')
    test_y = test_y.astype('float32')

    if resp_width != 0:
        std_y = std_y.ravel()
        mean_y = mean_y.ravel()
        train_y = train_y.reshape(train_y.shape[0], train_y.shape[1] * response)
        test_y = test_y.reshape(test_y.shape[0], test_y.shape[1] * response)
    else:
        pass

    print('train_X shape:', np.shape(train_X))
    print('train_y shape:', np.shape(train_y))
    print('test_X shape:', np.shape(test_X))
    print('test_y shape:', np.shape(test_y))

    return train_X, test_X, train_y, test_y, mean_x, std_x, mean_y, std_y

In [4]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true))


def compute_mape_minMax(model, x, y):
    forecasts = model.predict(x, batch_size=len(x))
    forecasts = (forecasts *  (max_soilT - min_soilT)) + min_soilT
    y_denom = (y * (max_soilT - min_soilT)) + min_soilT
    return np.mean(np.abs((y_denom - forecasts) / y_denom))


def compute_mape_centerScale(model, x, y):
    forecasts = model.predict(x)
#     forecasts = forecasts.reshape(forecasts.shape[0],dense)
    forecasts = (forecasts * std_y) + mean_y
    y_denom = (y * std_y) + mean_y
    sub_result = np.abs((y_denom - forecasts) / y_denom )
    # remove rows that are divided by 0 or a very small number
    idxinf = np.where(sub_result == np.inf)
    idx = np.where(sub_result > 1)
    total_remove = len(idxinf[0]) + len(idx[0])
    #print('Removed %f percent of the data, a total of %d rows.'% (((total_remove/len(forecasts))*100), total_remove))
    sub_result = np.delete(sub_result, idxinf[0],0)
    sub_result = np.delete(sub_result, idx[0], 0)
    result = (np.mean(sub_result))
    return result


def compute_mae(model, x, y):
    forecasts = model.predict(x)
#     forecasts = forecasts.reshape(forecasts.shape[0],dense)
    forecasts = (forecasts * std_y) + mean_y
    y_denom = (y * std_y) + mean_y
    result = mean_absolute_error(y_pred=forecasts, y_true=y_denom)
    return result


def test(model):
    # print('Train Mean Absolute Percentage Error (MAPE): {0:.3f}'.format(compute_mape_centerScale(model, train_X, train_y)))
    # print('Test Mean Absolute Percentage Error (MAPE): {0:.3f}'.format(compute_mape_centerScale(model, test_X, test_y)))
    return(compute_mape_centerScale(model, train_X, train_y), compute_mape_centerScale(model, test_X, test_y))

def test_mae(model):
    yhat_train = model.predict(train_X)
    yhat_test = model.predict(test_X)
    return(compute_mae(model, train_X, train_y), compute_mae(model, test_X, test_y))


def report(train, test):
    train_mean = np.asarray(train).mean()
    train_std = np.asarray(train).std()
    test_mean = np.asarray(test).mean()
    test_std = np.asarray(test).std()
    train = str('Training MAE %f ± %f'% (train_mean, train_std))
    test = str('Testing MAE %f ± %f'% (test_mean, test_std))
    return(train, test)

### Load Dataset

In [5]:
CWD = os.getcwd()
FILE_PATH = os.path.join(CWD, "../data/time_series_soilM_averaged_50percent_tuning.csv")
# FILE_PATH = os.path.join(CWD, "../data/time_series_soilM_averaged.csv")
dataset = pd.read_csv(FILE_PATH)
dataset.head()
dataset['date'] = pd.to_datetime(dataset['date'])
dataset = dataset.set_index(['date'])
dataset = dataset.reset_index()

,date,field,soilM_avg,imputed,precip.cm,tair.C,rh.pct,wind_sp.m_per_s,irradiance.w_per_m.2,average_sand,average_silt,average_clay
0,2014-04-14,AES,0.305329,0.305329,0.000,10.816667,39.958333,3.125000,252.708333,14.891866,60.796955,24.311179
1,2014-04-15,AES,0.307472,0.307472,0.000,8.850000,47.958333,8.145833,217.375000,14.891866,60.796955,24.311179
2,2014-04-16,AES,0.306836,0.306836,0.000,7.625000,67.125000,2.333333,130.291667,14.891866,60.796955,24.311179
3,2014-04-17,AES,0.305759,0.305759,1.106,7.920833,87.125000,3.370833,73.583333,14.891866,60.796955,24.311179
4,2014-04-18,AES,0.305109,0.305109,0.000,6.412500,65.958333,6.962500,252.916667,14.891866,60.796955,24.311179


In [6]:
dataset.columns

Index(['date', 'field', 'soilM_avg', 'imputed', 'precip.cm', 'tair.C',
       'rh.pct', 'wind_sp.m_per_s', 'irradiance.w_per_m.2', 'average_sand',
       'average_silt', 'average_clay'],
      dtype='object')

In [7]:
dataset.isnull().sum()

date                     0
field                    0
soilM_avg               18
imputed                  0
precip.cm                0
tair.C                   0
rh.pct                   0
wind_sp.m_per_s          0
irradiance.w_per_m.2     0
average_sand             0
average_silt             0
average_clay             0
dtype: int64

### Set Parameters

In [8]:
resp_cols = ['imputed']
feat_cols = ['precip.cm', 'tair.C', 'rh.pct', 'wind_sp.m_per_s', 'irradiance.w_per_m.2', 'average_clay',
       'average_sand', 'average_silt']
keep_cols = resp_cols + feat_cols

sequence_length = 30
horizon = 14
resp_width = 0
dense = len(resp_cols)

### Make Windowed DF

In [9]:
train_X, test_X, train_y, test_y, mean_x, std_x, mean_y, std_y = make_windowed(dataset=dataset, 
                                                                                     seq_length=sequence_length,
                                                                                     horizon=horizon,
                                                                                     resp_cols=resp_cols,
                                                                                     feat_cols=feat_cols,
                                                                                     region_col='field',
                                                                                     resp_width=resp_width,
                                                                                     predict_current=False)
dim = train_X.shape[2]

train_X shape: (2520, 30, 8)
train_y shape: (2520, 1)
test_X shape: (681, 30, 8)
test_y shape: (681, 1)


## Fit Model for Tuning

In [10]:
np.random.seed(84)
tf.set_random_seed(84)

def create_tCNN_model(max_meanPool_1=1,
                      max_meanPool_2=1,
                      max_meanPool_3=1,
                      max_meanPool_4=1,
                      max_meanPool_5=1,
                      max_meanPool_6=1,
                      filters1=32,
                      filters2=32,
                      filters3=32,
                      filters4=32,
                      filters5=32,
                      filters6=32,
                      filters_dense=32,
                      kernel_size1=1,
                      kernel_size2=1,
                      kernel_size3=1,
                      kernel_size4=1,
                      kernel_size5=1,
                      kernel_size6=1,
                      pool_size1=1,
                      pool_size2=1,
                      pool_size3=1,
                      pool_size4=1,
                      pool_size5=1,
                      pool_size6=1,
                      stride1=1,
                      stride2=1,
                      stride3=1,
                      stride4=1,
                      stride5=1,
                      stride6=1,
                      optimizer='RMSprop', 
                      layers=1,
                      init='glorot_normal',
                      lr=0.001, 
                      momentum=0.9,
                      decay=0.9,
                      beta_1=0.9,
                      beta_2=0.999,
                      rho=0.9,
                      nesterov=False,
                      activation = ELU(),
                      dropout_rate = 0.0,
                      epsilon=1e-08,
                      schedule_decay=0.004):
    
    sequence_input = Input(shape=(sequence_length, dim))
    if(layers == 1):
        x = Conv1D(filters1, kernel_size1)(sequence_input)
        x = activation(x)
        if(max_meanPool_1==1):
            x = MaxPooling1D(pool_size=pool_size1, stride=stride1)(x)
        elif(max_meanPool_1==2):  
            x = AveragePooling1D(pool_size=pool_size1, stride=stride1)(x)
        else:
            pass
        
    elif(layers == 2):
        x = Conv1D(filters1, kernel_size1, init=init)(sequence_input)
        x = activation(x)
        if(max_meanPool_1==1):
            x = MaxPooling1D(pool_size=pool_size1, stride=stride1)(x)
        elif(max_meanPool_1==2):  
            x = AveragePooling1D(pool_size=pool_size1, stride=stride1)(x)
        else:
            pass
        
        x = Conv1D(filters2, kernel_size2, init=init)(sequence_input)
        x = activation(x)
        if(max_meanPool_2==1):
            x = MaxPooling1D(pool_size=pool_size2, stride=stride2)(x)
        elif(max_meanPool_2==2):  
            x = AveragePooling1D(pool_size=pool_size2, stride=stride2)(x)
        else:
            pass
        
    elif(layers == 3):
        x = Conv1D(filters1, kernel_size1, init=init)(sequence_input)
        x = activation(x)
        if(max_meanPool_1==1):
            x = MaxPooling1D(pool_size=pool_size1, stride=stride1)(x)
        elif(max_meanPool_1==2):  
            x = AveragePooling1D(pool_size=pool_size1, stride=stride1)(x)
        else:
            pass
        
        x = Conv1D(filters2, kernel_size2, init=init)(sequence_input)
        x = activation(x)
        if(max_meanPool_2==1):
            x = MaxPooling1D(pool_size=pool_size2, stride=stride2)(x)
        elif(max_meanPool_2==2):  
            x = AveragePooling1D(pool_size=pool_size2, stride=stride2)(x)
        else:
            pass
        
        x = Conv1D(filters3, kernel_size3, init=init)(sequence_input)
        x = activation(x)
        if(max_meanPool_3==1):
            x = MaxPooling1D(pool_size=pool_size3, stride=stride3)(x)
        elif(max_meanPool_3==2):  
            x = AveragePooling1D(pool_size=pool_size3, stride=stride3)(x)
        else:
            pass
        
    elif(layers == 4):
        x = Conv1D(filters1, kernel_size1, init=init)(sequence_input)
        x = activation(x)
        if(max_meanPool_1==1):
            x = MaxPooling1D(pool_size=pool_size1, stride=stride1)(x)
        elif(max_meanPool_1==2):  
            x = AveragePooling1D(pool_size=pool_size1, stride=stride1)(x)
        else:
            pass
        
        x = Conv1D(filters2, kernel_size2, init=init)(sequence_input)
        x = activation(x)
        if(max_meanPool_2==1):
            x = MaxPooling1D(pool_size=pool_size2, stride=stride2)(x)
        elif(max_meanPool_2==2):  
            x = AveragePooling1D(pool_size=pool_size2, stride=stride2)(x)
        else:
            pass
        
        x = Conv1D(filters3, kernel_size3, init=init)(sequence_input)
        x = activation(x)
        if(max_meanPool_3==1):
            x = MaxPooling1D(pool_size=pool_size3, stride=stride3)(x)
        elif(max_meanPool_3==2):  
            x = AveragePooling1D(pool_size=pool_size3, stride=stride3)(x)
        else:
            pass
        
        x = Conv1D(filters4, kernel_size4, init=init)(sequence_input)
        x = activation(x)
        if(max_meanPool_4==1):
            x = MaxPooling1D(pool_size=pool_size4, stride=stride4)(x)
        elif(max_meanPool_4==2):  
            x = AveragePooling1D(pool_size=pool_size4, stride=stride4)(x)
        else:
            pass
        
    elif(layers == 5):
        x = Conv1D(filters1, kernel_size1, init=init)(sequence_input)
        x = activation(x)
        if(max_meanPool_1==1):
            x = MaxPooling1D(pool_size=pool_size1, stride=stride1)(x)
        elif(max_meanPool_1==2):  
            x = AveragePooling1D(pool_size=pool_size1, stride=stride1)(x)
        else:
            pass
        
        x = Conv1D(filters2, kernel_size2, init=init)(sequence_input)
        x = activation(x)
        if(max_meanPool_2==1):
            x = MaxPooling1D(pool_size=pool_size2, stride=stride2)(x)
        elif(max_meanPool_2==2):  
            x = AveragePooling1D(pool_size=pool_size2, stride=stride2)(x)
        else:
            pass
        
        x = Conv1D(filters3, kernel_size3, init=init)(sequence_input)
        x = activation(x)
        if(max_meanPool_3==1):
            x = MaxPooling1D(pool_size=pool_size3, stride=stride3)(x)
        elif(max_meanPool_3==2):  
            x = AveragePooling1D(pool_size=pool_size3, stride=stride3)(x)
        else:
            pass
        
        x = Conv1D(filters4, kernel_size4, init=init)(sequence_input)
        x = activation(x)
        if(max_meanPool_4==1):
            x = MaxPooling1D(pool_size=pool_size4, stride=stride4)(x)
        elif(max_meanPool_4==2):  
            x = AveragePooling1D(pool_size=pool_size4, stride=stride4)(x)
        else:
            pass
        
        x = Conv1D(filters5, kernel_size5, init=init)(sequence_input)
        x = activation(x)
        if(max_meanPool_5==1):
            x = MaxPooling1D(pool_size=pool_size5, stride=stride5)(x)
        elif(max_meanPool_5==2):  
            x = AveragePooling1D(pool_size=pool_size5, stride=stride5)(x)
        else:
            pass
        
    else:
        x = Conv1D(filters1, kernel_size1, init=init)(sequence_input)
        x = activation(x)
        if(max_meanPool_1==1):
            x = MaxPooling1D(pool_size=pool_size1, stride=stride1)(x)
        elif(max_meanPool_1==2):  
            x = AveragePooling1D(pool_size=pool_size1, stride=stride1)(x)
        else:
            pass
        
        x = Conv1D(filters2, kernel_size2, init=init)(sequence_input)
        x = activation(x)
        if(max_meanPool_2==1):
            x = MaxPooling1D(pool_size=pool_size2, stride=stride2)(x)
        elif(max_meanPool_2==2):  
            x = AveragePooling1D(pool_size=pool_size2, stride=stride2)(x)
        else:
            pass
        
        x = Conv1D(filters3, kernel_size3, init=init)(sequence_input)
        x = activation(x)
        if(max_meanPool_3==1):
            x = MaxPooling1D(pool_size=pool_size3, stride=stride3)(x)
        elif(max_meanPool_3==2):  
            x = AveragePooling1D(pool_size=pool_size3, stride=stride3)(x)
        else:
            pass
        
        x = Conv1D(filters4, kernel_size4, init=init)(sequence_input)
        x = activation(x)
        if(max_meanPool_4==1):
            x = MaxPooling1D(pool_size=pool_size4, stride=stride4)(x)
        elif(max_meanPool_4==2):  
            x = AveragePooling1D(pool_size=pool_size4, stride=stride4)(x)
        else:
            pass
        
        x = Conv1D(filters5, kernel_size5, init=init)(sequence_input)
        x = activation(x)
        if(max_meanPool_5==1):
            x = MaxPooling1D(pool_size=pool_size5, stride=stride5)(x)
        elif(max_meanPool_5==2):  
            x = AveragePooling1D(pool_size=pool_size5, stride=stride5)(x)
        else:
            pass
        
        x = Conv1D(filters6, kernel_size6, init=init)(sequence_input)
        x = activation(x)
        if(max_meanPool_6==1):
            x = MaxPooling1D(pool_size=pool_size6, stride=stride6)(x)
        elif(max_meanPool_6==2):  
            x = AveragePooling1D(pool_size=pool_size6, stride=stride6)(x)
        else:
            pass
        
    x = Flatten()(x)
    x = Dense(filters_dense, init=init)(x)
    x = activation(x)
    x = Dropout(dropout_rate)(x)
    preds = Dense(dense)(x)
    model = Model(sequence_input, preds)

    if(optimizer == 'SGD'):
        sgd = SGD(lr=lr, momentum=momentum, decay=decay, nesterov=nesterov)
    elif(optimizer == 'RMSprop'):
        sgd = RMSprop(lr=lr, rho=rho)
    elif(optimizer == 'Nadam'):
        sgd = Nadam(lr=lr, beta_1=beta_1, beta_2=beta_2, epsilon=epsilon, schedule_decay=schedule_decay)
    elif(optimizer == 'Adagrad'):
        sgd = Adagrad(lr=lr, epsilon=epsilon, decay=decay)
    elif(optimizer == 'Adadelta'):
        sgd = Adadelta(lr=lr, rho=rho, epsilon=epsilon, decay=decay)
    elif(optimizer == 'Adam'):
        sgd = Adam(lr=lr, beta_1=beta_1, beta_2=beta_2, epsilon=epsilon, decay=decay)
    elif(optimizer == 'Adamax'):
        sgd = Adamax(lr=lr, beta_1=beta_1, beta_2=beta_2, epsilon=epsilon, decay=decay)
    else:
        pass
    
    model.compile(loss='mae', optimizer=sgd, metrics=['mse'])
    
    return model

In [13]:
print('Forecasting out {0} days, with a {1:0.1f} day window. '.format(horizon, (sequence_length)))
print('Feature space: ', feat_cols)
print('Response: ', resp_cols)

model = KerasRegressor(build_fn=create_tCNN_model, shuffle=True, verbose=0)

# init = ['uniform', 'normal', 'zero', 'glorot_uniform', 'glorot_normal', 'lecun_uniform', 
#                       'lecun_normal', 'he_normal', 'he_uniform']
# optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']

layers = [3]
nesterov = [False]

filters1 = [64]
filters2 = [32, 64, 128, 256, 512]
filters3 = [32, 64, 128, 256, 512]
filters4 = [32]
filters5 = [32]
filters6 = [32]
filters_dense = [32, 64, 128, 256, 512]

kernel_size1 = [3]
kernel_size2 = [1,2,3,4]
kernel_size3 = [1,2,3,4]
kernel_size4 = [1]
kernel_size5 = [1]
kernel_size6 = [1]

pool_size1 = [4] 
pool_size2 = [1,2,3,4] 
pool_size3 = [1,2,3,4] 
pool_size4 = [1] 
pool_size5 = [1] 
pool_size6 = [1]

stride1 = [3]
stride2 = [1,2,3,4]
stride3 = [1,2,3,4]
stride4 = [1]
stride5 = [1]
stride6 = [1]

max_meanPool_1=[2]
max_meanPool_2=[1,2]
max_meanPool_3=[1,2]
max_meanPool_4=[2]
max_meanPool_5=[2]
max_meanPool_6=[2]

batch_size = [1,2,3,4,5,6]
# batch_size = [64]
epochs = [1]
# lr = [10**k for k in range(-5, -1)]
lr=[0.00015, 0.0001, 0.0005, 0.001]

optimizer = ['RMSprop']
init = ['lecun_normal']

momentum=[0.9]
dropout_rate=[0.0]
# decay = np.arange(0,.1,0.001).tolist()
decay = [0.012]
# decay = np.arange(0.06,0.1,0.001).tolist()
# beta_1 = np.random.uniform(low=0.90, high=1.0, size=20).tolist()
# beta_2 = np.random.uniform(0.90, 1.0, 20).tolist()
beta_1 = [0.9130588049258307]
beta_2 = [0.9498343369399377]
# rho = np.random.uniform(0.80, 0.95, 20).tolist()
rho = [0.8026518983228177]
# epsilon = [1e-09, 1e-08,1e-07, 1e-06, 1e-05, 1e-04]
epsilon = [1e-09]
# schedule_decay = np.arange(0.001,0.01,0.0005).tolist()
schedule_decay=[0.0055]
param_grid = dict(batch_size=batch_size, 
                  epsilon=epsilon,
                  schedule_decay=schedule_decay,
                  epochs=epochs, 
                  layers=layers, 
                  filters1=filters1,
                  filters2=filters2,
                  filters3=filters3,
                  filters4=filters4,
                  filters5=filters5,
                  filters6=filters6,
                  filters_dense=filters_dense,
                  stride1=stride1,
                  stride2=stride2,
                  stride3=stride3,
                  stride4=stride4,
                  stride5=stride5,
                  stride6=stride6,
                  pool_size1=pool_size1,
                  pool_size2=pool_size2,
                  pool_size3=pool_size3,
                  pool_size4=pool_size4,
                  pool_size5=pool_size5,
                  pool_size6=pool_size6,
                  kernel_size1=kernel_size1,
                  kernel_size2=kernel_size2,
                  kernel_size3=kernel_size3,
                  kernel_size4=kernel_size4,
                  kernel_size5=kernel_size5,
                  kernel_size6=kernel_size6,
                  optimizer=optimizer,
                  lr=lr,
                  decay=decay,
                  init=init,
                  beta_1=beta_1,
                  beta_2=beta_2,
                  nesterov=nesterov,
                  rho=rho,
                  momentum=momentum,
                  dropout_rate=dropout_rate,
                  max_meanPool_1=max_meanPool_1, 
                  max_meanPool_2=max_meanPool_2, 
                  max_meanPool_3=max_meanPool_3, 
                  max_meanPool_4=max_meanPool_4, 
                  max_meanPool_5=max_meanPool_5,
                  max_meanPool_6=max_meanPool_6)

start_timing()
sweeper = random_sweep(
    train_X, train_y, 
    model, param_grid,
    scoring=compute_mae, 
    n_iter=1, n_jobs=3, 
    refit=False, cv=3, verbose=1)
report_timing()


Forecasting out 14 days, with a 30.0 day window. 
Feature space:  ['precip.cm', 'tair.C', 'rh.pct', 'wind_sp.m_per_s', 'irradiance.w_per_m.2', 'average_clay', 'average_sand', 'average_silt']
Response:  ['imputed']
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Elapsed: 0.13 minutes


[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    7.5s finished


In [14]:
results = sweep_stats(sweeper)
# FILE_PATH = os.path.join(CWD, "../data/results_data/best_params/tCNN_W30_H14_Resp-soilM_Feat-soilType-allWeather_fineTune_run2.csv")
# results.to_csv(FILE_PATH)
results

,Score,Std,batch_size,beta_1,beta_2,decay,dropout_rate,epochs,epsilon,filters1,...,pool_size5,pool_size6,rho,schedule_decay,stride1,stride2,stride3,stride4,stride5,stride6
0,1.594888,2.191362,6,0.913059,0.949834,0.012,0.0,1,1.000000e-09,64,...,1,1,0.802652,0.0055,3,4,3,1,1,1


In [18]:
for i in range(1):

    RUN = i+1
    print('Best Run Num: {0}'.format(RUN))
    params = results.iloc[-RUN,2:].to_dict()
    params
    params['kernel_size1'] = int(params['kernel_size1'])
    params['kernel_size2'] = int(params['kernel_size2'])
    params['kernel_size3'] = int(params['kernel_size3'])
    params['kernel_size4'] = int(params['kernel_size4'])
    params['kernel_size5'] = int(params['kernel_size5'])
    params['kernel_size6'] = int(params['kernel_size6'])

    params['pool_size1'] = int(params['pool_size1'])
    params['pool_size2'] = int(params['pool_size2'])
    params['pool_size3'] = int(params['pool_size3'])
    params['pool_size4'] = int(params['pool_size4'])
    params['pool_size5'] = int(params['pool_size5'])
    params['pool_size6'] = int(params['pool_size6'])

    params['stride1'] = int(params['stride1'])
    params['stride2'] = int(params['stride2'])
    params['stride3'] = int(params['stride3'])
    params['stride4'] = int(params['stride4'])
    params['stride5'] = int(params['stride5'])
    params['stride6'] = int(params['stride6'])

    params['dropout_rate'] = params.pop('dropout_rate')
    ### Testing the best params

    np.random.seed(84)
    tf.set_random_seed(84)

    params['epochs'] = 5

    train_results, test_results = [], []
    ITERATIONS=1
    VERBOSE=1

    # params

    # early_stopping = EarlyStopping(monitor='val_loss', patience=10)
#     reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=1e-08, mode='auto', verbose=1)

    for i in range(ITERATIONS):
        model = KerasRegressor(build_fn=create_tCNN_model, shuffle=True, verbose=VERBOSE, **params)
    #     history = model.fit(train_X, train_y, verbose=VERBOSE, validation_split = 0.15, callbacks=[early_stopping])
        history = model.fit(train_X, train_y, verbose=VERBOSE, validation_split = None)
        train_results.append(test_mae(model)[0])
        test_results.append(test_mae(model)[1])
    report(train_results, test_results)

Best Run Num: 1


{'batch_size': 6,
 'beta_1': 0.9130588049258307,
 'beta_2': 0.9498343369399377,
 'decay': 0.012,
 'dropout_rate': 0.0,
 'epochs': 1,
 'epsilon': 1e-09,
 'filters1': 64,
 'filters2': 512,
 'filters3': 256,
 'filters4': 32,
 'filters5': 32,
 'filters6': 32,
 'filters_dense': 256,
 'init': 'lecun_normal',
 'kernel_size1': 3,
 'kernel_size2': 4,
 'kernel_size3': 1,
 'kernel_size4': 1,
 'kernel_size5': 1,
 'kernel_size6': 1,
 'layers': 3,
 'lr': 0.001,
 'max_meanPool_1': 2,
 'max_meanPool_2': 1,
 'max_meanPool_3': 1,
 'max_meanPool_4': 2,
 'max_meanPool_5': 2,
 'max_meanPool_6': 2,
 'momentum': 0.9,
 'nesterov': False,
 'optimizer': 'RMSprop',
 'pool_size1': 4,
 'pool_size2': 4,
 'pool_size3': 1,
 'pool_size4': 1,
 'pool_size5': 1,
 'pool_size6': 1,
 'rho': 0.8026518983228177,
 'schedule_decay': 0.0055,
 'stride1': 3,
 'stride2': 4,
 'stride3': 3,
 'stride4': 1,
 'stride5': 1,
 'stride6': 1}

Epoch 1/5
5244/5244 [==============================] - 4s 714us/step - loss: 1.0637 - mean_squared_error: 4.2698
Epoch 2/5
5244/5244 [==============================] - 3s 653us/step - loss: 0.5078 - mean_squared_error: 0.4609
Epoch 3/5
5244/5244 [==============================] - 3s 642us/step - loss: 0.4827 - mean_squared_error: 0.4190
Epoch 4/5
5244/5244 [==============================] - 3s 642us/step - loss: 0.4680 - mean_squared_error: 0.3974
Epoch 5/5
1364/1364 [==============================] - 0s 167us/step


('Training MAE 0.029693 ± 0.000000', 'Testing MAE 0.048434 ± 0.000000')

In [22]:
RUN = 6

print('Best Run Num: {0}'.format(RUN))
params = results.iloc[-RUN,2:].to_dict()
params
params['kernel_size1'] = int(params['kernel_size1'])
params['kernel_size2'] = int(params['kernel_size2'])
params['kernel_size3'] = int(params['kernel_size3'])
params['kernel_size4'] = int(params['kernel_size4'])
params['kernel_size5'] = int(params['kernel_size5'])
params['kernel_size6'] = int(params['kernel_size6'])

params['pool_size1'] = int(params['pool_size1'])
params['pool_size2'] = int(params['pool_size2'])
params['pool_size3'] = int(params['pool_size3'])
params['pool_size4'] = int(params['pool_size4'])
params['pool_size5'] = int(params['pool_size5'])
params['pool_size6'] = int(params['pool_size6'])

params['stride1'] = int(params['stride1'])
params['stride2'] = int(params['stride2'])
params['stride3'] = int(params['stride3'])
params['stride4'] = int(params['stride4'])
params['stride5'] = int(params['stride5'])
params['stride6'] = int(params['stride6'])

params['dropout_rate'] = params.pop('dropout_rate')
### Testing the best params

np.random.seed(84)
tf.set_random_seed(84)

params['epochs'] = 150

train_results, test_results = [], []
ITERATIONS=1
VERBOSE=1

# params

# early_stopping = EarlyStopping(monitor='val_loss', patience=10)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=15, min_lr=1e-08, mode='auto', verbose=1)

for i in range(ITERATIONS):
    model = KerasRegressor(build_fn=create_tCNN_model, shuffle=True, verbose=VERBOSE, **params)
    history = model.fit(train_X, train_y, verbose=VERBOSE, validation_split = 0.15, callbacks=[reduce_lr])
#     history = model.fit(train_X, train_y, verbose=VERBOSE, validation_split = None)
    train_results.append(test_mae(model)[0])
    test_results.append(test_mae(model)[1])
report(train_results, test_results)

Best Run Num: 6


{'batch_size': 3,
 'beta_1': 0.9130588049258307,
 'beta_2': 0.9498343369399377,
 'decay': 0.012,
 'dropout_rate': 0.0,
 'epochs': 40,
 'epsilon': 1e-09,
 'filters1': 64,
 'filters2': 128,
 'filters3': 256,
 'filters4': 32,
 'filters5': 32,
 'filters6': 32,
 'filters_dense': 256,
 'init': 'lecun_normal',
 'kernel_size1': 3,
 'kernel_size2': 4,
 'kernel_size3': 4,
 'kernel_size4': 1,
 'kernel_size5': 1,
 'kernel_size6': 1,
 'layers': 3,
 'lr': 0.001,
 'max_meanPool_1': 2,
 'max_meanPool_2': 2,
 'max_meanPool_3': 1,
 'max_meanPool_4': 2,
 'max_meanPool_5': 2,
 'max_meanPool_6': 2,
 'momentum': 0.9,
 'nesterov': False,
 'optimizer': 'RMSprop',
 'pool_size1': 4,
 'pool_size2': 1,
 'pool_size3': 3,
 'pool_size4': 1,
 'pool_size5': 1,
 'pool_size6': 1,
 'rho': 0.8026518983228177,
 'schedule_decay': 0.0055,
 'stride1': 3,
 'stride2': 2,
 'stride3': 1,
 'stride4': 1,
 'stride5': 1,
 'stride6': 1}

Train on 4457 samples, validate on 787 samples
Epoch 1/150
4457/4457 [==============================] - 23s 5ms/step - loss: 0.6623 - mean_squared_error: 4.1181 - val_loss: 1.6089 - val_mean_squared_error: 3.2941
Epoch 2/150
4457/4457 [==============================] - 19s 4ms/step - loss: 0.4821 - mean_squared_error: 0.3904 - val_loss: 0.6475 - val_mean_squared_error: 0.6752
Epoch 3/150
4457/4457 [==============================] - 19s 4ms/step - loss: 0.4494 - mean_squared_error: 0.3488 - val_loss: 0.8719 - val_mean_squared_error: 1.2019
Epoch 4/150
4457/4457 [==============================] - 19s 4ms/step - loss: 0.4366 - mean_squared_error: 0.3228 - val_loss: 1.0578 - val_mean_squared_error: 1.5612
Epoch 5/150
4457/4457 [==============================] - 19s 4ms/step - loss: 0.4128 - mean_squared_error: 0.2825 - val_loss: 0.8305 - val_mean_squared_error: 1.1066
Epoch 6/150
4457/4457 [==============================] - 20s 5ms/step - loss: 0.4025 - mean_squared_error: 0.2651 - val_los

Epoch 49/150
4443/4457 [============================>.] - ETA: 0s - loss: 0.0877 - mean_squared_error: 0.0132
Epoch 00049: ReduceLROnPlateau reducing learning rate to 4.0000001899898055e-05.
4457/4457 [==============================] - 18s 4ms/step - loss: 0.0878 - mean_squared_error: 0.0132 - val_loss: 0.7372 - val_mean_squared_error: 0.8883
Epoch 50/150
4457/4457 [==============================] - 18s 4ms/step - loss: 0.0574 - mean_squared_error: 0.0063 - val_loss: 0.7647 - val_mean_squared_error: 0.9306
Epoch 51/150
4457/4457 [==============================] - 19s 4ms/step - loss: 0.0530 - mean_squared_error: 0.0056 - val_loss: 0.7404 - val_mean_squared_error: 0.8916
Epoch 52/150
4457/4457 [==============================] - 18s 4ms/step - loss: 0.0506 - mean_squared_error: 0.0052 - val_loss: 0.7213 - val_mean_squared_error: 0.8574
Epoch 53/150
4457/4457 [==============================] - 18s 4ms/step - loss: 0.0491 - mean_squared_error: 0.0050 - val_loss: 0.7472 - val_mean_squared_e

Epoch 95/150
4457/4457 [==============================] - 17s 4ms/step - loss: 0.0264 - mean_squared_error: 0.0025 - val_loss: 0.7463 - val_mean_squared_error: 0.8986
Epoch 96/150
4457/4457 [==============================] - 17s 4ms/step - loss: 0.0264 - mean_squared_error: 0.0025 - val_loss: 0.7455 - val_mean_squared_error: 0.8975
Epoch 97/150
4457/4457 [==============================] - 17s 4ms/step - loss: 0.0263 - mean_squared_error: 0.0025 - val_loss: 0.7456 - val_mean_squared_error: 0.8975
Epoch 98/150
4457/4457 [==============================] - 17s 4ms/step - loss: 0.0263 - mean_squared_error: 0.0025 - val_loss: 0.7463 - val_mean_squared_error: 0.8987
Epoch 99/150
4457/4457 [==============================] - 18s 4ms/step - loss: 0.0263 - mean_squared_error: 0.0025 - val_loss: 0.7461 - val_mean_squared_error: 0.8985
Epoch 100/150
4457/4457 [==============================] - 17s 4ms/step - loss: 0.0263 - mean_squared_error: 0.0025 - val_loss: 0.7464 - val_mean_squared_error: 0.89

Epoch 141/150
4457/4457 [==============================] - 19s 4ms/step - loss: 0.0261 - mean_squared_error: 0.0025 - val_loss: 0.7461 - val_mean_squared_error: 0.8983
Epoch 142/150
4457/4457 [==============================] - 17s 4ms/step - loss: 0.0261 - mean_squared_error: 0.0025 - val_loss: 0.7461 - val_mean_squared_error: 0.8984
Epoch 143/150
4457/4457 [==============================] - 17s 4ms/step - loss: 0.0261 - mean_squared_error: 0.0025 - val_loss: 0.7461 - val_mean_squared_error: 0.8983
Epoch 144/150
4457/4457 [==============================] - 22s 5ms/step - loss: 0.0261 - mean_squared_error: 0.0025 - val_loss: 0.7461 - val_mean_squared_error: 0.8984
Epoch 145/150
4457/4457 [==============================] - 17s 4ms/step - loss: 0.0261 - mean_squared_error: 0.0025 - val_loss: 0.7461 - val_mean_squared_error: 0.8984
Epoch 146/150
4457/4457 [==============================] - 17s 4ms/step - loss: 0.0261 - mean_squared_error: 0.0025 - val_loss: 0.7461 - val_mean_squared_error:

('Training MAE 0.009258 ± 0.000000', 'Testing MAE 0.048564 ± 0.000000')

In [19]:
from keras.models import load_model
model.model.save('../data/results_data/models/CNN_dailyALLDATA_W30_H14_FULLMODEL.h5')
model.model.save_weights('../data/results_data/models/CNN_dailyALLDATA_W30_H14_WEIGHTS_ONLY.h5')

In [31]:
model.model.save

,Score,Std,batch_size,beta_1,beta_2,decay,dropout_rate,epochs,epsilon,filters1,...,pool_size5,pool_size6,rho,schedule_decay,stride1,stride2,stride3,stride4,stride5,stride6
0,15.884001,7.216298,3,0.913059,0.949834,0.012,0.0,40,1.000000e-09,64,...,1,1,0.802652,0.0055,3,3,4,1,1,1
1,7.824623,7.865014,4,0.913059,0.949834,0.012,0.0,40,1.000000e-09,64,...,1,1,0.802652,0.0055,3,1,1,1,1,1
2,1.507169,2.063936,5,0.913059,0.949834,0.012,0.0,40,1.000000e-09,64,...,1,1,0.802652,0.0055,3,3,3,1,1,1
3,1.473696,2.027749,3,0.913059,0.949834,0.012,0.0,40,1.000000e-09,64,...,1,1,0.802652,0.0055,3,1,1,1,1,1
4,1.335490,1.819293,1,0.913059,0.949834,0.012,0.0,40,1.000000e-09,64,...,1,1,0.802652,0.0055,3,3,1,1,1,1
5,1.225948,1.662097,3,0.913059,0.949834,0.012,0.0,40,1.000000e-09,64,...,1,1,0.802652,0.0055,3,2,3,1,1,1
6,1.181798,1.610822,1,0.913059,0.949834,0.012,0.0,40,1.000000e-09,64,...,1,1,0.802652,0.0055,3,3,4,1,1,1
7,1.068585,1.455542,6,0.913059,0.949834,0.012,0.0,40,1.000000e-09,64,...,1,1,0.802652,0.0055,3,4,3,1,1,1
8,1.004185,1.191110,6,0.913059,0.949834,0.012,0.0,40,1.000000e-09,64,...,1,1,0.802652,0.0055,3,1,2,1,1,1
9,0.961936,1.283685,6,0.913059,0.949834,0.012,0.0,40,1.000000e-09,64,...,1,1,0.802652,0.0055,3,4,2,1,1,1
